In [1]:
import numpy as np
import pandas as pd
from misc import load, load_class_labels
from sklearn.metrics import classification_report

predictions_path = "predictions/"
test_info = load("../dataset/labels/test.csv")
class_labels = load_class_labels('../dataset/classes.txt')

def load_predictions(path, model_name):
    predictions = np.load(path + model_name + ".npy")
    names = pd.read_csv(path + model_name + "_names.csv")['filename'].values
    indices = np.argsort(names)
    return predictions[indices], names[indices]

2024-01-15 17:49:13.938811: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 17:49:13.938876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 17:49:13.994174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 17:49:14.111367: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 17:49:15.152472: W tensorflow/compiler/tf2

In [2]:
vit_predictions, names = load_predictions(predictions_path, "ViT_extended_restored_limited")
en_predictions, _ = load_predictions(predictions_path, "efficientnet_v2_extended_restored_limited")

test_names = test_info['filename'].values
test_labels = test_info['label'].values

ordered_indices = np.argsort(test_names)
ordered_labels = test_labels[ordered_indices]

filtered_info = test_info[test_info['filename'].isin(names)]
filtered_info.sort_values(by='filename', inplace=True)
ordered_labels = filtered_info['label'].values

final_prob = np.zeros(vit_predictions.shape)
for i, (vit_pred, en_pred) in enumerate(zip(vit_predictions, en_predictions)):
    final_prob[i] = np.sqrt(vit_pred * en_pred)
    final_prob[i] /= final_prob[i].sum()
    
final_predictions = np.argmax(final_prob, axis=1)

/tmp/ipykernel_4865/2451221489.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_info.sort_values(by='filename', inplace=True)


In [3]:
from tensorflow import keras


print(classification_report(ordered_labels, final_predictions, target_names=class_labels))

                               precision    recall  f1-score   support

                      macaron       0.90      0.88      0.89        50
                      beignet       0.67      0.86      0.75        58
                      cruller       0.78      0.69      0.73        52
                  cockle_food       0.60      0.70      0.65        50
                       samosa       0.62      0.87      0.72        39
                     tiramisu       0.69      0.65      0.67        51
                      tostada       0.70      0.58      0.63        55
                     moussaka       0.51      0.64      0.57        53
                     dumpling       0.58      0.63      0.61        49
                      sashimi       0.69      0.74      0.71        42
                        knish       0.86      0.55      0.67        44
                    croquette       0.44      0.64      0.52        45
                     couscous       0.31      0.59      0.41        44
     

/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [4]:
def top_k(predictions, gt, k = 1):
    ohe = keras.utils.to_categorical(gt, num_classes=251)
    m = keras.metrics.TopKCategoricalAccuracy(k)
    m.update_state(ohe, predictions)
    return m.result().numpy()

print(top_k(final_prob, ordered_labels, 3))
print(top_k(final_prob, ordered_labels, 1))

2024-01-15 17:49:17.033644: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-15 17:49:17.238898: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-15 17:49:17.239177: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

0.80074894
0.62570757


2024-01-15 17:49:19.130463: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23057864 exceeds 10% of free system memory.
